In [ ]:
import tensorflow as tf
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/dataset/train'

# Create datasets with a 20-80 train-validation split
train_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Normalize the pixel values
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Define data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),           # Random rotations up to 20%
    tf.keras.layers.RandomWidth(0.2),              # Random horizontal shifts
    tf.keras.layers.RandomHeight(0.2),             # Random vertical shifts
    tf.keras.layers.RandomZoom(0.2),               # Random zooms
    tf.keras.layers.RandomFlip("horizontal"),      # Random horizontal flips
])

# Preprocess training data with augmentation
def preprocess_with_augmentation(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values
    image = data_augmentation(image)  # Apply augmentation
    return image, label

# Apply preprocessing and augmentation to the training data
train_dataset = train_dataset.map(preprocess_with_augmentation)

# Apply preprocessing (without augmentation) to the validation data
val_dataset = val_dataset.map(preprocess)

# Optimize for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Print confirmation
print("Data augmentation applied to the training dataset!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 518 files belonging to 3 classes.
Using 415 files for training.
Found 518 files belonging to 3 classes.
Using 103 files for validation.
Data augmentation applied to the training dataset!


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (VGG19,DenseNet201, MobileNetV2,InceptionResNetV2)

def build_and_train_model(base_model, train_dataset, val_dataset, num_classes=4, model_name=""):
    # Freeze the base model
    base_model.trainable = False

    # Build the model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=50,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )

    # Evaluate
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"{model_name} - Validation Accuracy: {val_accuracy:.2f}")

    return model, history


In [ ]:
from tensorflow.keras.applications import (VGG19, InceptionResNetV2, DenseNet201,MobileNetV2)

# Define models to train
pretrained_models = [
    ("InceptionResNetV2", InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
]

# Storage for trained models and results
trained_models = {}
model_histories = {}
results = []

# Train each model
for model_name, base_model in pretrained_models:
    print(f"Training {model_name}...")
    model, history = build_and_train_model(base_model, train_dataset, val_dataset, model_name=model_name)
    trained_models[model_name] = model
    model_histories[model_name] = history
    val_loss, val_accuracy = model.evaluate(val_dataset)
    results.append((model_name, val_accuracy))

Training InceptionResNetV2...
Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 227s 15s/step - accuracy: 0.3560 - loss: 1.7133 - val_accuracy: 0.4660 - val_loss: 1.0537
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 58s 220ms/step - accuracy: 0.5324 - loss: 1.0514 - val_accuracy: 0.5534 - val_loss: 0.9030
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.5138 - loss: 0.9543 - val_accuracy: 0.5922 - val_loss: 0.8561
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - accuracy: 0.5778 - loss: 0.9091 - val_accuracy: 0.5049 - val_loss: 1.1630
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.5541 - loss: 1.0125 - val_accuracy: 0.5922 - val_loss: 0.8510
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.6087 - loss: 0.8150 - val_accuracy: 0.6408 - val_loss: 0.8140
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 194ms/step - accuracy: 0.7249 - loss: 0.7051 - val_accuracy: 0.6408 - val_loss: 0.8199
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 185ms/step - accuracy: 0.6744 - 

In [ ]:
# Fine-tune each model
fine_tuned_results = []

for model_name, base_model in pretrained_models:
    print(f"Fine-tuning {model_name}...")
    trained_model = trained_models[model_name]
    base_model.trainable = True  # Unfreeze base model

    # Compile with lower learning rate
    trained_model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune the model
    fine_tune_history = trained_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=20,
        callbacks=[
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )

    # Evaluate fine-tuned model
    val_loss, val_accuracy = trained_model.evaluate(val_dataset)
    fine_tuned_results.append((model_name, val_accuracy))
    print(f"{model_name} - Fine-tuned Validation Accuracy: {val_accuracy:.2f}")


Fine-tuning InceptionResNetV2...
Epoch 1/20
12/13 ━━━━━━━━━━━━━━━━━━━━ 1:05 66s/step - accuracy: 0.4636 - loss: 1.0245

In [ ]:
import numpy as np

# Gather predictions from fine-tuned models
predictions = []
for model_name in trained_models:
    print(f"Predicting with {model_name}...")
    model = trained_models[model_name]
    preds = model.predict(val_dataset)
    predictions.append(preds)

# Average predictions across models
ensemble_preds = np.mean(predictions, axis=0)

# Convert ensemble predictions to class labels
final_preds = np.argmax(ensemble_preds, axis=1)

# Evaluate ensemble performance
from sklearn.metrics import accuracy_score

y_true = np.concatenate([y for x, y in val_dataset], axis=0)  # True labels
ensemble_accuracy = accuracy_score(y_true, final_preds)
print(f"Ensemble Model Accuracy: {ensemble_accuracy:.2f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications
import (VGG19, InceptionResNetV2, DenseNet201, MobileNetV2,)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

# Function to plot ROC curves
def plot_roc_curve(models, val_dataset, n_classes=4):
    model_roc_data = {}

    # Iterate over each model
    for model_name, model in models:
        print(f"Evaluating {model_name}...")

        # Get true labels and predictions
        y_true = np.concatenate([y for _, y in val_dataset], axis=0)
        y_pred_proba = model.predict(val_dataset)

        # One-hot encode true labels
        y_true_bin = label_binarize(y_true, classes=range(n_classes))

        # Compute ROC curve and AUC
        fpr, tpr, roc_auc = {}, {}, {}
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Micro-average ROC curve
        fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Store results for plotting
        model_roc_data[model_name] = (fpr, tpr, roc_auc)

    # Plot ROC curves for all models
    plt.figure(figsize=(12, 10))
    colors = cycle(["blue", "red", "green", "orange", "purple", "cyan", "magenta", "brown", "grey", "pink"])
    for model_name, color in zip(model_roc_data.keys(), colors):
        fpr, tpr, roc_auc = model_roc_data[model_name]

        # Plot micro-average
        plt.plot(
            fpr["micro"], tpr["micro"], color=color, linestyle="--", lw=2,
            label=f"{model_name} (AUC = {roc_auc['micro']:.2f})"
        )

    plt.plot([0, 1], [0, 1], "k--", lw=2)  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate", fontsize=14)
    plt.ylabel("True Positive Rate", fontsize=14)
    plt.title("ROC Curve for Multiple Models", fontsize=16)
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(alpha=0.3)
    plt.show()

# Example of training models
models_to_train = [
    ("InceptionResNetV2", InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
]


# Placeholder for trained models and histories
trained_models = []
model_histories = []

for model_name, base_model in models_to_train:
    print(f"Training {model_name}...")
    model, history = build_and_train_model(base_model, train_dataset, val_dataset, model_name=model_name)
    trained_models.append((model_name, model))
    model_histories.append(history)

# Plot Loss/Accuracy for all models
plot_loss_accuracy_graphs(model_histories, [model_name for model_name, _ in models_to_train])

# Plot ROC curve
plot_roc_curve(trained_models, val_dataset, n_classes=4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_loss_accuracy_graphs(models_history, model_names):
    """
    Plots Loss and Accuracy for multiple models in one graph.

    Arguments:
    models_history : list of histories - training histories of the models.
    model_names : list of model names - names for each model.
    """
    plt.figure(figsize=(14, 6))

    # Subplot 1: Accuracy Graph
    plt.subplot(1, 2, 1)
    for history, name in zip(models_history, model_names):
        plt.plot(history.history['accuracy'], label=f'{name} - Train')
        plt.plot(history.history['val_accuracy'], label=f'{name} - Validation')
    plt.title('Accuracy for Models')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

    # Subplot 2: Loss Graph
    plt.subplot(1, 2, 2)
    for history, name in zip(models_history, model_names):
        plt.plot(history.history['loss'], label=f'{name} - Train')
        plt.plot(history.history['val_loss'], label=f'{name} - Validation')
    plt.title('Loss for Models')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Assuming you have a list of trained models and their histories
models_history = [history_inceptionresnetv2]
model_names = ["InceptionResNetV2"]

# Call the function to plot loss and accuracy graphs
plot_loss_accuracy_graphs(models_history, model_names)


In [ ]:
def plot_roc_curve_separate(models, val_dataset, n_classes=4):
    """
    Plot separate ROC curves for each model.
    """
    for model_name, model in models:
        print(f"Evaluating {model_name}...")

        # Get true labels and predictions
        y_true = np.concatenate([y for _, y in val_dataset], axis=0)
        y_pred_proba = model.predict(val_dataset)

        # One-hot encode true labels
        y_true_bin = label_binarize(y_true, classes=range(n_classes))

        # Compute ROC curve and AUC
        fpr, tpr, roc_auc = {}, {}, {}
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Micro-average ROC curve
        fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot ROC curve for the current model
        plt.figure(figsize=(8, 6))
        colors = cycle(["blue", "red", "green", "orange"])

        for i, color in zip(range(n_classes), colors):
            plt.plot(
                fpr[i], tpr[i], color=color, lw=2,
                label=f"Class {i} (AUC = {roc_auc[i]:.2f})"
            )

        # Plot micro-average
        plt.plot(
            fpr["micro"], tpr["micro"], color="purple", linestyle="--", lw=2,
            label=f"Micro-average (AUC = {roc_auc['micro']:.2f})"
        )

        plt.plot([0, 1], [0, 1], "k--", lw=2)  # Diagonal line
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate", fontsize=14)
        plt.ylabel("True Positive Rate", fontsize=14)
        plt.title(f"ROC Curve for {model_name}", fontsize=16)
        plt.legend(loc="lower right", fontsize=12)
        plt.grid(alpha=0.3)
        plt.show()

# Plot ROC curve for each model separately
plot_roc_curve_separate(trained_models, val_dataset, n_classes=4)


In [ ]:
import matplotlib.pyplot as plt

def plot_loss_and_accuracy_separately(models_history, model_names):
    """
    Plots separate Loss and Accuracy graphs for each model.

    Arguments:
    models_history : list of histories - training histories of the models.
    model_names : list of model names - names for each model.
    """
    for history, name in zip(models_history, model_names):
        # Plot Accuracy
        plt.figure(figsize=(8, 6))
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Accuracy for {name}')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

        # Plot Loss
        plt.figure(figsize=(8, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Loss for {name}')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()

# Assuming you have a list of trained models and their histories
models_history = [history_vgg19, history_inceptionresnetv2, history_densenet201, history_mobilenetv2]
model_names = ["VGG19", "InceptionResNetV2", "DenseNet201", "MobileNetV2"]

# Call the function to plot separate graphs
plot_loss_and_accuracy_separately(models_history, model_names)
